In [6]:
import pandas as pd
import os
import datetime
import inflection
import glob

In [7]:
os.getcwd()

'/home/jian/Projects/Big_Lots/Browsing_Path/Search_terms'

In [8]:
file_list=glob.glob('/home/jian/Projects/Big_Lots/Browsing_Path/Search_terms/'+"*.csv")
writer=pd.ExcelWriter("/home/jian/Projects/Big_Lots/Browsing_Path/Search_terms/output/BL_search_terms_coount_by_page_"+str(datetime.datetime.now().date())+".xlsx")

In [9]:
for i in range(6):
    rewards_file=[x for x in file_list if "_Rewards_" in x and "Page_"+str(i) in x][0]
    non_rewards_file=[x for x in file_list if "_NonRewards_" in x and "Page_"+str(i) in x][0]
    
    rewards_data=pd.read_csv(rewards_file,skiprows=25,dtype=str).iloc[:,1:]
    rewards_data.columns=['search_term','rewards_count','rewards_pctg']
    rewards_data['rewards_count']=rewards_data['rewards_count'].astype(int)
    rewards_data['search_term']=rewards_data['search_term'].apply(lambda x: inflection.singularize(x))
    rewards_data['search_term']=rewards_data['search_term'].apply(lambda x: x.lower())
    rewards_data=rewards_data.groupby(['search_term'])['rewards_count'].sum().to_frame().reset_index()
    total_rewards_searched=rewards_data['rewards_count'].sum()
    rewards_data['rewards_pctg']=rewards_data['rewards_count'].apply(lambda x: float(x/total_rewards_searched),6)

    non_rewards_data=pd.read_csv(non_rewards_file,skiprows=25,dtype=str).iloc[:,1:]
    non_rewards_data.columns=['search_term','Non_rewards_count','Non_rewards_pctg']
    non_rewards_data['Non_rewards_count']=non_rewards_data['Non_rewards_count'].astype(int)
    non_rewards_data['search_term']=non_rewards_data['search_term'].apply(lambda x: inflection.singularize(x))
    non_rewards_data['search_term']=non_rewards_data['search_term'].apply(lambda x: x.lower())
    non_rewards_data=non_rewards_data.groupby(['search_term'])['Non_rewards_count'].sum().to_frame().reset_index()
    total_Non_rewards_searched=non_rewards_data['Non_rewards_count'].sum()
    non_rewards_data['Non_rewards_pctg']=non_rewards_data['Non_rewards_count'].apply(lambda x: float(x/total_Non_rewards_searched),6)
    
    output=pd.merge(rewards_data,non_rewards_data,on="search_term",how="outer")
    output=output.fillna(0)
    output=output.sort_values(['rewards_count','Non_rewards_count'],ascending=False)
    output.to_excel(writer,"Page_"+str(i),index=False)
    
    
    

In [10]:
writer.save()